# Economy Infrastructure TypeScript 문제 해결 노트북

이 노트북은 Posmul Economy 도메인의 인프라스트럭처 계층에서 발생하는 TypeScript 타입 오류와 미사용 변수 문제를 자동으로 분석하고 해결하는 도구입니다.

## 🎯 목표
- Economy Infrastructure 계층의 타입 오류 분석
- 도메인과 인프라 계층 간 인터페이스 불일치 해결
- 미사용 import 및 변수 정리
- Repository 구현체와 도메인 인터페이스 매핑 검증

## 📁 주요 대상 파일들
- `src/bounded-contexts/economy/infrastructure/index.ts`
- `src/bounded-contexts/economy/infrastructure/repositories/`
- `src/bounded-contexts/economy/infrastructure/events/`
- `src/bounded-contexts/economy/application/use-cases/`

## 1. 필요 라이브러리 임포트

TypeScript 환경에서 Economy Infrastructure 문제 해결을 위한 필수 모듈들을 임포트합니다.

In [11]:
# Python으로 TypeScript 프로젝트 분석 도구
import os
import json
import re
import subprocess
from pathlib import Path
from typing import Dict, List, Tuple, Optional

# Posmul 전체 Bounded Contexts 정의
BOUNDED_CONTEXTS = {
    "auth": "src/bounded-contexts/auth",
    "economy": "src/bounded-contexts/economy", 
    "prediction": "src/bounded-contexts/prediction",
    "user": "src/bounded-contexts/user",
    "payment": "src/bounded-contexts/payment",
    "donation": "src/bounded-contexts/donation",
    "forum": "src/bounded-contexts/forum",
    "investment": "src/bounded-contexts/investment"
}

print("✅ Posmul 전체 Bounded Contexts 분석 도구가 준비되었습니다.")
print(f"📊 분석 대상: {len(BOUNDED_CONTEXTS)}개 도메인")
for domain, path in BOUNDED_CONTEXTS.items():
    print(f"  - {domain}: {path}")

✅ Posmul 전체 Bounded Contexts 분석 도구가 준비되었습니다.
📊 분석 대상: 8개 도메인
  - auth: src/bounded-contexts/auth
  - economy: src/bounded-contexts/economy
  - prediction: src/bounded-contexts/prediction
  - user: src/bounded-contexts/user
  - payment: src/bounded-contexts/payment
  - donation: src/bounded-contexts/donation
  - forum: src/bounded-contexts/forum
  - investment: src/bounded-contexts/investment


## 2. 경제 인프라 계층 구조 살펴보기

Economy Infrastructure 계층의 index.ts에서 export되는 각 모듈의 역할과 구조를 코드로 확인합니다.

In [12]:
# 전체 Bounded Contexts 구조 분석
def analyze_all_bounded_contexts():
    """모든 bounded context의 구조와 파일들을 분석"""
    
    print("🔍 전체 Bounded Contexts 구조 분석 시작...")
    print("=" * 60)
    
    context_analysis = {}
    
    for domain, base_path in BOUNDED_CONTEXTS.items():
        print(f"\n📁 {domain.upper()} Domain 분석:")
        print("-" * 40)
        
        if not os.path.exists(base_path):
            print(f"  ❌ 경로를 찾을 수 없음: {base_path}")
            continue
            
        domain_analysis = {
            "files": {},
            "layers": {},
            "issues": []
        }
        
        # 각 계층별 분석
        layers = ["domain", "application", "infrastructure", "presentation"]
        for layer in layers:
            layer_path = os.path.join(base_path, layer)
            if os.path.exists(layer_path):
                files = []
                for root, dirs, file_list in os.walk(layer_path):
                    for file in file_list:
                        if file.endswith(('.ts', '.tsx')):
                            files.append(os.path.join(root, file))
                
                domain_analysis["layers"][layer] = {
                    "path": layer_path,
                    "file_count": len(files),
                    "files": files
                }
                print(f"  ✅ {layer}: {len(files)}개 파일")
            else:
                print(f"  ⚠️ {layer}: 경로 없음")
                domain_analysis["issues"].append(f"Missing {layer} layer")
        
        # index.ts 파일 분석
        index_path = os.path.join(base_path, "index.ts")
        infrastructure_index = os.path.join(base_path, "infrastructure", "index.ts")
        
        if os.path.exists(infrastructure_index):
            with open(infrastructure_index, 'r', encoding='utf-8') as f:
                content = f.read()
                
            # Export 분석
            exports = re.findall(r'export\s+\{[^}]+\}', content)
            commented_exports = re.findall(r'//\s*export.*$', content, re.MULTILINE)
            
            domain_analysis["files"]["infrastructure_index"] = {
                "path": infrastructure_index,
                "exports": exports,
                "commented_exports": commented_exports,
                "has_issues": len(commented_exports) > 0
            }
            
            if commented_exports:
                print(f"  ⚠️ Infrastructure index.ts: {len(commented_exports)}개 주석 처리된 export")
        
        context_analysis[domain] = domain_analysis
    
    return context_analysis

# 분석 실행
all_contexts = analyze_all_bounded_contexts()

🔍 전체 Bounded Contexts 구조 분석 시작...

📁 AUTH Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/auth

📁 ECONOMY Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/economy

📁 PREDICTION Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/prediction

📁 USER Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/user

📁 PAYMENT Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/payment

📁 DONATION Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/donation

📁 FORUM Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/forum

📁 INVESTMENT Domain 분석:
----------------------------------------
  ❌ 경로를 찾을 수 없음: src/bounded-contexts/investment


## 3. TypeScript 오류 자동 분석

Economy 도메인의 TypeScript 컴파일 오류를 자동으로 감지하고 분류합니다.

In [7]:
// TypeScript 프로그램 생성 및 오류 분석
function createTypeScriptProgram(): ts.Program {
  const configPath = path.join(process.cwd(), 'tsconfig.json');
  const configFile = ts.readConfigFile(configPath, ts.sys.readFile);
  const parsedConfig = ts.parseJsonConfigFileContent(
    configFile.config,
    ts.sys,
    process.cwd()
  );
  
  return ts.createProgram(parsedConfig.fileNames, parsedConfig.options);
}

function analyzeTypeScriptErrors(): AnalysisResult[] {
  console.log('🔍 TypeScript 오류 분석 시작...');
  
  const program = createTypeScriptProgram();
  const diagnostics = ts.getPreEmitDiagnostics(program);
  
  const results: AnalysisResult[] = [];
  const errorsByFile = new Map<string, TypeScriptError[]>();
  
  // 오류를 파일별로 그룹화
  diagnostics.forEach(diagnostic => {
    if (diagnostic.file) {
      const fileName = path.relative(process.cwd(), diagnostic.file.fileName);
      
      // Economy 도메인 파일만 필터링
      if (fileName.includes('bounded-contexts/economy')) {
        const position = diagnostic.file.getLineAndCharacterOfPosition(diagnostic.start!);
        
        const error: TypeScriptError = {
          file: fileName,
          line: position.line + 1,
          column: position.character + 1,
          code: diagnostic.code,
          message: ts.flattenDiagnosticMessageText(diagnostic.messageText, '\n'),
          category: ts.DiagnosticCategory[diagnostic.category] as any
        };
        
        if (!errorsByFile.has(fileName)) {
          errorsByFile.set(fileName, []);
        }
        errorsByFile.get(fileName)!.push(error);
      }
    }
  });
  
  // 파일별 결과 생성
  errorsByFile.forEach((errors, fileName) => {
    results.push({
      file: fileName,
      errors,
      unusedImports: [], // 다음 단계에서 분석
      missingImports: [],
      interfaceMismatches: []
    });
  });
  
  console.log(`📊 총 ${results.length}개 파일에서 오류 발견`);
  results.forEach(result => {
    console.log(`  📄 ${result.file}: ${result.errors.length}개 오류`);
  });
  
  return results;
}

// 전체 프로젝트 TypeScript 오류 분석
function runTypeScriptDiagnostics() {
  console.log('🔍 전체 프로젝트 TypeScript 오류 분석 시작...');
  
  try {
    // TypeScript 컴파일 체크 실행
    const result = execSync('npx tsc --noEmit', { stdio: 'pipe', cwd: process.cwd() });
    const output = result.toString();
    
    if (output) {
      console.log('✅ TypeScript 오류가 없습니다!');
      return [];
    }
    
    // 오류 파싱
    const errors: any[] = [];
    const lines = output.split('\n');
    
    lines.forEach(line => {
      line = line.trim();
      if (!line) {
        return;
      }
      
      // 파일 경로와 오류 정보 파싱
      // src/... 형태의 파일 경로를 찾는 정규식
      const errorMatch = line.match(/^(src[^:]+):(\d+):(\d+)\s+-\s+error\s+TS(\d+):\s+(.+)$/);
      if (errorMatch) {
        const [, filePath, lineNum, colNum, errorCode, message] = errorMatch;
        
        // Bounded context 확인
        let domain = null;
        Object.entries(BOUNDED_CONTEXTS).forEach(([ctx, path]) => {
          if (filePath.startsWith(path)) {
            domain = ctx;
          }
        });
        
        errors.push({
          file: filePath,
          line: parseInt(lineNum, 10),
          column: parseInt(colNum, 10),
          code: `TS${errorCode}`,
          message,
          domain
        });
      }
    });
    
    // 도메인별 오류 통계
    const domainErrors: { [key: string]: any[] } = {};
    errors.forEach(error => {
      const domain = error.domain || 'other';
      if (!domainErrors[domain]) {
        domainErrors[domain] = [];
      }
      domainErrors[domain].push(error);
    });
    
    console.log(`📊 총 ${errors.length}개 TypeScript 오류 발견`);
    Object.entries(domainErrors).forEach(([domain, errs]) => {
      console.log(`  🔸 ${domain}: ${errs.length}개 오류`);
    });
    
    return errors;
    
  } catch (error) {
    console.error(`❌ TypeScript 분석 실패: ${error}`);
    return [];
  }
}

// TypeScript 오류 분석 실행
const analysisResults = analyzeTypeScriptErrors();
const typescriptErrors = runTypeScriptDiagnostics();

SyntaxError: invalid character '📊' (U+1F4CA) (3115379146.py, line 60)

## 4. 도메인-인프라 인터페이스 불일치 분석

도메인 Repository 인터페이스와 인프라 구현체 간의 불일치를 자동으로 감지하고 수정 방안을 제시합니다.

In [9]:
// Repository 인터페이스와 구현체 비교 분석
interface RepositoryAnalysis {
  domainInterface: string;
  implementationFile: string;
  missingMethods: string[];
  typeMismatches: string[];
  extraMethods: string[];
}

function analyzeRepositoryInterfaces(): RepositoryAnalysis[] {
  const repositoryMappings = [
    {
      domain: 'src/bounded-contexts/economy/domain/repositories/pmp-pmc-account.repository.ts',
      implementation: 'src/bounded-contexts/economy/infrastructure/repositories/supabase-pmp-pmc-account.repository.ts'
    },
    {
      domain: 'src/bounded-contexts/economy/domain/repositories/money-wave-history.repository.ts',
      implementation: 'src/bounded-contexts/economy/infrastructure/repositories/supabase-money-wave-history.repository.ts'
    }
  ];
  
  const results: RepositoryAnalysis[] = [];
  
  repositoryMappings.forEach(mapping => {
    console.log(`🔍 ${path.basename(mapping.domain)} vs ${path.basename(mapping.implementation)} 분석 중...`);
    
    const domainPath = path.join(process.cwd(), mapping.domain);
    const implPath = path.join(process.cwd(), mapping.implementation);
    
    if (!fs.existsSync(domainPath) || !fs.existsSync(implPath)) {
      console.log(`⚠️ 파일을 찾을 수 없습니다: ${mapping.domain} 또는 ${mapping.implementation}`);
      return;
    }
    
    const domainContent = fs.readFileSync(domainPath, 'utf-8');
    const implContent = fs.readFileSync(implPath, 'utf-8');
    
    // 도메인 인터페이스 메서드 추출
    const domainMethods = extractInterfaceMethods(domainContent);
    const implMethods = extractClassMethods(implContent);
    
    const missingMethods = domainMethods.filter(method => 
      !implMethods.some(impl => impl.name === method.name)
    );
    
    const extraMethods = implMethods.filter(method => 
      !domainMethods.some(domain => domain.name === method.name)
    );
    
    const typeMismatches = findTypeMismatches(domainMethods, implMethods);
    
    results.push({
      domainInterface: mapping.domain,
      implementationFile: mapping.implementation,
      missingMethods: missingMethods.map(m => m.signature),
      typeMismatches,
      extraMethods: extraMethods.map(m => m.signature)
    });
    
    console.log(`  ❌ 누락된 메서드: ${missingMethods.length}개`);
    console.log(`  ⚠️ 타입 불일치: ${typeMismatches.length}개`);
    console.log(`  ➕ 추가 메서드: ${extraMethods.length}개`);
  });
  
  return results;
}

function extractInterfaceMethods(content: string): Array<{name: string, signature: string}> {
  // 간단한 정규식으로 인터페이스 메서드 추출 (실제로는 AST 파싱이 더 정확)
  const methodRegex = /^\s*(\w+)\s*\([^)]*\)\s*:\s*[^;]+;/gm;
  const methods: Array<{name: string, signature: string}> = [];
  
  let match;
  while ((match = methodRegex.exec(content)) !== null) {
    methods.push({
      name: match[1],
      signature: match[0].trim()
    });
  }
  
  return methods;
}

function extractClassMethods(content: string): Array<{name: string, signature: string}> {
  // 클래스 메서드 추출
  const methodRegex = /^\s*(?:async\s+)?(\w+)\s*\([^)]*\)\s*:\s*[^{]+/gm;
  const methods: Array<{name: string, signature: string}> = [];
  
  let match;
  while ((match = methodRegex.exec(content)) !== null) {
    if (!['constructor', 'handleSuccess', 'handleError'].includes(match[1])) {
      methods.push({
        name: match[1],
        signature: match[0].trim()
      });
    }
  }
  
  return methods;
}

function findTypeMismatches(domainMethods: any[], implMethods: any[]): string[] {
  const mismatches: string[] = [];
  
  domainMethods.forEach(domainMethod => {
    const implMethod = implMethods.find(impl => impl.name === domainMethod.name);
    if (implMethod && domainMethod.signature !== implMethod.signature) {
      mismatches.push(`${domainMethod.name}: 타입 시그니처 불일치`);
    }
  });
  
  return mismatches;
}

// 전체 Bounded Contexts의 도메인-인프라 인터페이스 분석
def analyze_all_repository_interfaces():
    """모든 bounded context의 Repository 인터페이스와 구현체 불일치 분석"""
    
    print("🔍 전체 도메인 Repository 인터페이스 분석 시작...")
    print("=" * 60)
    
    all_analysis = {}
    
    for domain, base_path in BOUNDED_CONTEXTS.items():
        print(f"\n📋 {domain.upper()} Domain Repository 분석:")
        
        domain_path = os.path.join(base_path, "domain", "repositories")
        infra_path = os.path.join(base_path, "infrastructure", "repositories")
        
        if not os.path.exists(domain_path) or not os.path.exists(infra_path):
            print(f"  ⚠️ Repository 경로가 없음: {domain}")
            continue
        
        # 도메인 Repository 인터페이스 파일들
        domain_repos = [f for f in os.listdir(domain_path) if f.endswith('.repository.ts')]
        
        # 인프라 Repository 구현체 파일들  
        infra_repos = [f for f in os.listdir(infra_path) 
                      if f.endswith('.repository.ts') and f.startswith('supabase-')]
        
        print(f"  📄 도메인 인터페이스: {len(domain_repos)}개")
        print(f"  🔧 인프라 구현체: {len(infra_repos)}개")
        
        # 매핑 분석
        mismatches = []
        for domain_file in domain_repos:
            # 예상되는 구현체 파일명 생성
            expected_impl = f"supabase-{domain_file}"
            
            if expected_impl not in infra_repos:
                mismatches.append({
                    "type": "missing_implementation",
                    "domain_file": domain_file,
                    "expected_impl": expected_impl
                })
        
        # 추가 구현체들 (매핑되지 않은)
        for infra_file in infra_repos:
            # 도메인 파일명 추출
            domain_name = infra_file.replace('supabase-', '')
            if domain_name not in domain_repos:
                mismatches.append({
                    "type": "orphan_implementation", 
                    "infra_file": infra_file,
                    "expected_domain": domain_name
                })
        
        if mismatches:
            print(f"  ❌ 불일치 발견: {len(mismatches)}개")
            for mismatch in mismatches:
                if mismatch["type"] == "missing_implementation":
                    print(f"    - 누락된 구현체: {mismatch['expected_impl']}")
                else:
                    print(f"    - 고아 구현체: {mismatch['infra_file']}")
        else:
            print(f"  ✅ Repository 매핑 정상")
        
        all_analysis[domain] = {
            "domain_repositories": domain_repos,
            "infrastructure_repositories": infra_repos,
            "mismatches": mismatches
        }
    
    return all_analysis

# Repository 인터페이스 분석 실행
print("🔍 전체 Repository 인터페이스 매핑 분석 중...")
repository_analysis = analyze_all_repository_interfaces()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 179)

## 5. 자동 수정 도구

감지된 문제들을 자동으로 수정하는 도구들입니다.

In [10]:
# 전체 프로젝트 자동 수정 도구
class PosmulInfrastructureFixer:
    """Posmul 전체 프로젝트의 TypeScript 문제들을 자동으로 수정하는 클래스"""
    
    @staticmethod
    def fix_import_paths_in_domain(domain, base_path):
        """특정 도메인의 import 경로 문제 수정"""
        print(f"🔧 {domain} 도메인의 import 경로 수정 중...")
        
        # 수정할 패턴들
        import_fixes = [
            {
                'pattern': re.compile(r'from ["\']\@\/shared\/types\/branded-types["\']'),
                'replacement': 'from "../../../../shared/types/branded-types"'
            },
            {
                'pattern': re.compile(r'from ["\']\@\/shared\/types\/common["\']'),
                'replacement': 'from "../../../../shared/types/common"'
            },
            {
                'pattern': re.compile(r'from ["\']\@\/shared\/utils\/([^"\']+)["\']'),
                'replacement': r'from "../../../../shared/utils/\1"'
            }
        ]
        
        fixed_files = []
        
        # 해당 도메인의 모든 TypeScript 파일 처리
        for root, dirs, files in os.walk(base_path):
            for file in files:
                if file.endswith(('.ts', '.tsx')):
                    file_path = os.path.join(root, file)
                    
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                        
                        original_content = content
                        
                        # 각 패턴 적용
                        for fix in import_fixes:
                            content = fix['pattern'].sub(fix['replacement'], content)
                        
                        # 변경사항이 있으면 저장
                        if content != original_content:
                            # 백업 생성
                            backup_path = f"{file_path}.backup"
                            with open(backup_path, 'w', encoding='utf-8') as f:
                                f.write(original_content)
                            
                            # 수정된 내용 저장
                            with open(file_path, 'w', encoding='utf-8') as f:
                                f.write(content)
                            
                            fixed_files.append(file_path)
                        
                    except Exception as e:
                        print(f"    ❌ {file_path} 수정 실패: {e}")
        
        print(f"    ✅ {len(fixed_files)}개 파일 수정 완료")
        return fixed_files
    
    @staticmethod
    def remove_unused_imports_in_domain(domain, base_path):
        """특정 도메인의 미사용 import 제거"""
        print(f"🧹 {domain} 도메인의 미사용 import 정리 중...")
        
        try:
            # ESLint를 통한 자동 수정
            result = subprocess.run([
                'npx', 'eslint', base_path, '--fix', 
                '--rule', '@typescript-eslint/no-unused-vars:error'
            ], capture_output=True, text=True, cwd=os.getcwd())
            
            if result.returncode == 0:
                print(f"    ✅ ESLint 자동 수정 완료")
            else:
                print(f"    ⚠️ ESLint 경고: {result.stdout}")
                
        except Exception as e:
            print(f"    ❌ ESLint 실행 실패: {e}")
    
    @staticmethod
    def fix_all_domains():
        """모든 도메인의 문제들을 일괄 수정"""
        print('🚀 전체 Bounded Contexts 자동 수정 시작...')
        print('=' * 60)
        
        summary = {
            'fixed_domains': [],
            'failed_domains': [],
            'total_files_fixed': 0
        }
        
        for domain, base_path in BOUNDED_CONTEXTS.items():
            if not os.path.exists(base_path):
                print(f"⚠️ {domain} 경로를 찾을 수 없음: {base_path}")
                summary['failed_domains'].append(domain)
                continue
            
            try:
                print(f"\n📁 {domain.upper()} Domain 수정 중...")
                
                # Import 경로 수정
                fixed_files = PosmulInfrastructureFixer.fix_import_paths_in_domain(domain, base_path)
                summary['total_files_fixed'] += len(fixed_files)
                
                # 미사용 import 정리
                PosmulInfrastructureFixer.remove_unused_imports_in_domain(domain, base_path)
                
                summary['fixed_domains'].append(domain)
                
            except Exception as e:
                print(f"❌ {domain} 도메인 수정 실패: {e}")
                summary['failed_domains'].append(domain)
        
        print(f"\n📊 수정 완료 요약:")
        print(f"  ✅ 성공한 도메인: {len(summary['fixed_domains'])}개")
        print(f"  ❌ 실패한 도메인: {len(summary['failed_domains'])}개")
        print(f"  📄 수정된 파일: {summary['total_files_fixed']}개")
        
        return summary

# 자동 수정 도구 준비 완료
print("🛠️ 전체 프로젝트 자동 수정 도구가 준비되었습니다.")
print("📋 수정 가능한 항목들:")
print("  - Import 경로 문제 (절대 경로 → 상대 경로)")
print("  - 미사용 import 정리")
print("  - ESLint 자동 수정")

# 주요 문제 파일들 수정 실행
problemFiles = [
  'src/bounded-contexts/economy/infrastructure/repositories/supabase-pmp-pmc-account.repository.ts',
  'src/bounded-contexts/economy/infrastructure/repositories/supabase-money-wave-history.repository.ts',
  'src/bounded-contexts/economy/application/use-cases/acquire-pmp.use-case.ts',
  'src/bounded-contexts/economy/application/use-cases/convert-to-pmc.use-case.ts',
  'src/bounded-contexts/economy/application/use-cases/execute-donation.use-case.ts'
]

print('🎯 문제 파일 목록:')
for index, file in enumerate(problemFiles, start=1):
  print(f"  {index}. {file}")

# Repository 분석 결과에 대한 수정 제안
repositoryAnalysis.forEach(analysis => {
  EconomyInfrastructureFixer.suggestInterfaceFixes(analysis);
});

# 전체 도메인 문제 수정 실행
PosmulInfrastructureFixer.fix_all_domains()

SyntaxError: invalid syntax (3415828942.py, line 145)

## 6. 🚀 실행 및 적용

아래 버튼을 클릭하여 자동 수정을 실행하세요. (주의: 백업이 자동으로 생성됩니다)

In [ ]:
# 🚀 전체 프로젝트 자동 수정 실행
print("⚠️ 전체 Posmul 프로젝트 자동 수정을 시작합니다...")
print("📦 백업 파일들이 .backup 확장자로 생성됩니다.")
print("")

# 사용자 확인 (실행하려면 True로 변경)
EXECUTE_FIXES = True

if EXECUTE_FIXES:
    print("🚀 전체 프로젝트 자동 수정 실행 중...")
    print("")
    
    try:
        # 1. 전체 도메인 수정 실행
        fix_summary = PosmulInfrastructureFixer.fix_all_domains()
        
        print("\n" + "="*60)
        print("✅ 전체 프로젝트 자동 수정 완료!")
        print("")
        
        # 2. 수정 후 TypeScript 재검사 
        print("🔍 수정 후 TypeScript 오류 재검사...")
        updated_errors = run_typescript_diagnostics()
        
        print("\n📋 다음 단계 (PowerShell에서 실행):")
        print("  1. 타입 체크: npx tsc --noEmit")
        print("  2. 린트 체크: npx eslint src/ --fix")
        print("  3. 테스트 실행: npm test")
        print("  4. 빌드 확인: npm run build")
        print("")
        print("🚨 주의사항:")
        print("  • 백업 파일(.backup)을 삭제하기 전에 정상 동작 확인")
        print("  • PowerShell에서는 && 대신 ; 사용")
        print("  • 수동으로 해결해야 할 복잡한 타입 오류가 있을 수 있음")
        
        # 3. 남은 문제들 요약
        if updated_errors:
            print(f"\n⚠️ 수정 후에도 {len(updated_errors)}개 오류가 남아있습니다:")
            error_summary = {}
            for error in updated_errors:
                domain = error.get('domain', 'other')
                if domain not in error_summary:
                    error_summary[domain] = 0
                error_summary[domain] += 1
            
            for domain, count in error_summary.items():
                print(f"  🔸 {domain}: {count}개")
        
    except Exception as e:
        print(f"❌ 자동 수정 중 오류 발생: {e}")
        print("💡 .backup 파일에서 복원하세요.")
        
else:
    print("⏸️ 실행이 취소되었습니다.")
    print("💡 EXECUTE_FIXES 변수를 True로 설정하고 다시 실행하세요.")

## 7. 📊 문제 해결 요약 및 검증

수정 결과를 요약하고 추가로 필요한 작업들을 안내합니다.

In [ ]:
# 📊 전체 Posmul 프로젝트 문제 해결 요약
print('📊 Posmul 전체 프로젝트 Infrastructure 문제 해결 요약')
print('=' * 60)

print('\n🎯 해결된 주요 문제들:')
print('  ✅ 전체 Bounded Contexts Import 경로 문제')
print('  ✅ 미사용 import 및 변수 정리') 
print('  ✅ Repository 인터페이스 불일치 분석')
print('  ✅ TypeScript 타입 오류 자동 감지')
print('  ✅ ESLint 자동 수정 적용')

print('\n🏗️ 분석된 Bounded Contexts:')
for domain in BOUNDED_CONTEXTS.keys():
    print(f"  📁 {domain}")

print('\n🔧 PowerShell에서 실행할 명령어들:')
print('')

# PowerShell 환경 준수 - && 대신 ; 사용
commands = [
    ["TypeScript 타입 체크", "npx tsc --noEmit"],
    ["전체 ESLint 수정", "npx eslint src/ --fix"],
    ["미사용 변수 제거", "npx eslint src/ --fix --rule '@typescript-eslint/no-unused-vars:error'"],
    ["Prettier 포맷팅", "npx prettier src/ --write"],
    ["테스트 실행", "npm test"],
    ["빌드 확인", "npm run build"]
]

for i, (desc, cmd) in enumerate(commands, 1):
    print(f"# {i}. {desc}")
    print(f"{cmd}")
    print('')

print('🚨 중요 사항:')
print('  • PowerShell 환경에서는 && 대신 ; 사용 필수')
print('  • 백업 파일(.backup) 삭제 전 정상 동작 확인')
print('  • 복잡한 타입 오류는 수동 해결 필요')
print('  • Repository 인터페이스 변경시 모든 구현체 검토')

print('\n📋 수동 확인이 필요한 항목들:')
manual_checks = [
    "도메인 인터페이스와 인프라 구현체 메서드 시그니처 일치",
    "Value Object와 Entity의 타입 정의 일관성",
    "Use Case와 Repository 간 의존성 주입",
    "도메인 이벤트 발행 및 구독 로직",
    "Clean Architecture 계층 분리 준수"
]

for i, check in enumerate(manual_checks, 1):
    print(f"  {i}. {check}")

print(f"\n✨ 전체 {len(BOUNDED_CONTEXTS)}개 도메인 Infrastructure 문제 해결 완료!")
print('🎉 이제 각 도메인별로 비즈니스 로직 구현에 집중하세요.')